In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense # type: ignore
from tensorflow.keras.models import Model #type: ignore
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv('data_with_nifty.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,start_pin,destination_pin,travel_distance,quantity_tons,period,nifty_infra_price,amount
0,110092,392001,1115,75.0,November 2023,6585.6,457500.00
1,124106,141015,367,13.0,October 2023,6095.4,81453.13
2,124106,143006,493,12.0,November 2023,6585.6,47120.62
3,124106,147001,304,40.0,October 2023,6095.4,212500.00
4,124146,140306,306,39.4,October 2023,6095.4,233938.00


### I am trying to add new features to the model

1. distance_per_ton - can be used to measure the efficiency of the route
2. route_frequency - define the popularity of a particular route
3. avg_route_price - average price of the route


In [4]:
df['distance_per_ton'] = df['travel_distance'] / df['quantity_tons']
df['distance_per_ton'] = df['distance_per_ton'].replace([np.inf, -np.inf], np.nan)
df['distance_per_ton'] = df['distance_per_ton'].fillna(df['distance_per_ton'].mean())

In [5]:
route_frequency = df.groupby(['start_pin', 'destination_pin']).size().reset_index(name='frequency')
df = df.merge(route_frequency, on=['start_pin', 'destination_pin'], how='left')
df['frequency'] = df['frequency'].fillna(0) 

In [6]:
avg_route_price = df.groupby(['start_pin', 'destination_pin'])['amount'].mean().reset_index(name='avg_route_price')
df = df.merge(avg_route_price, on=['start_pin', 'destination_pin'], how='left')
df['avg_route_price'] = df['avg_route_price'].fillna(df['amount'].mean()) 

In [7]:
df.columns

Index(['start_pin', 'destination_pin', 'travel_distance', 'quantity_tons',
       'period', 'nifty_infra_price', 'amount', 'distance_per_ton',
       'frequency', 'avg_route_price'],
      dtype='object')

In [8]:
df.head()

,start_pin,destination_pin,travel_distance,quantity_tons,period,nifty_infra_price,amount,distance_per_ton,frequency,avg_route_price
0,110092,392001,1115,75.0,November 2023,6585.6,457500.00,14.866667,1,457500.0000
1,124106,141015,367,13.0,October 2023,6095.4,81453.13,28.230769,3,49967.8700
2,124106,143006,493,12.0,November 2023,6585.6,47120.62,41.083333,1,47120.6200
3,124106,147001,304,40.0,October 2023,6095.4,212500.00,7.600000,4,226862.3775
4,124146,140306,306,39.4,October 2023,6095.4,233938.00,7.766497,4,221755.2500


In [9]:
def check_infinite_values(df, columns):
    for col in columns:
        inf_count = np.isinf(df[col]).sum()
        nan_count = np.isnan(df[col]).sum()
        if inf_count > 0 or nan_count > 0:
            print(f"Column {col}: Inf count = {inf_count}, NaN count = {nan_count}")

In [10]:
le_start = LabelEncoder()
le_dest = LabelEncoder()
df['start_pin'] = le_start.fit_transform(df['start_pin'])
df['destination_pin'] = le_dest.fit_transform(df['destination_pin'])

In [11]:
df = df.rename(columns={'frequency': 'route_frequency'})

In [15]:
df = df.rename(columns={'quantity_tons': 'Quantity (In TON)'})

In [16]:
df.head()

,start_pin,destination_pin,travel_distance,Quantity (In TON),period,nifty_infra_price,amount,distance_per_ton,route_frequency,avg_route_price
0,30,2349,1115,75.0,November 2023,6585.6,457500.00,14.866667,1,457500.0000
1,42,233,367,13.0,October 2023,6095.4,81453.13,28.230769,3,49967.8700
2,42,257,493,12.0,November 2023,6585.6,47120.62,41.083333,1,47120.6200
3,42,325,304,40.0,October 2023,6095.4,212500.00,7.600000,4,226862.3775
4,43,218,306,39.4,October 2023,6095.4,233938.00,7.766497,4,221755.2500


In [58]:
X = df.drop(columns=['amount'])
y = df['amount']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=324)

In [19]:
df

,start_pin,destination_pin,travel_distance,Quantity (In TON),period,nifty_infra_price,amount,distance_per_ton,route_frequency,avg_route_price
0,30,2349,1115,75.00,November 2023,6585.60,457500.00,14.866667,1,4.575000e+05
1,42,233,367,13.00,October 2023,6095.40,81453.13,28.230769,3,4.996787e+04
2,42,257,493,12.00,November 2023,6585.60,47120.62,41.083333,1,4.712062e+04
3,42,325,304,40.00,October 2023,6095.40,212500.00,7.600000,4,2.268624e+05
4,43,218,306,39.40,October 2023,6095.40,233938.00,7.766497,4,2.217552e+05
...,...,...,...,...,...,...,...,...,...,...
31077,442,2293,625,35.79,June 2023,5738.70,216417.84,17.462978,1,2.164178e+05
31078,442,2319,653,29.75,June 2023,5738.70,172921.88,21.949580,1,1.729219e+05
31079,449,1919,399,42.00,July 2023,6115.35,1023758.40,9.500000,1,1.023758e+06
31080,464,1877,399,30.00,July 2023,6115.35,131250.00,13.300000,3,1.526042e+05


In [45]:
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

In [46]:
df.head()

,start_pin,destination_pin,travel_distance,Quantity (In TON),period,nifty_infra_price,amount,distance_per_ton,route_frequency,avg_route_price,period_dt,year,month,month_sin,month_cos
0,30,2349,1115,75.0,November 2023,6585.6,457500.00,14.866667,1,457500.0000,2023-11-01,2023,11,-0.500000,0.866025
1,42,233,367,13.0,October 2023,6095.4,81453.13,28.230769,3,49967.8700,2023-10-01,2023,10,-0.866025,0.500000
2,42,257,493,12.0,November 2023,6585.6,47120.62,41.083333,1,47120.6200,2023-11-01,2023,11,-0.500000,0.866025
3,42,325,304,40.0,October 2023,6095.4,212500.00,7.600000,4,226862.3775,2023-10-01,2023,10,-0.866025,0.500000
4,43,218,306,39.4,October 2023,6095.4,233938.00,7.766497,4,221755.2500,2023-10-01,2023,10,-0.866025,0.500000


In [20]:
num_features = [
    'travel_distance', 
    'Quantity (In TON)', 
    'distance_per_ton', 
    'route_frequency',
    'avg_route_price',
    'nifty_infra_price'
]
check_infinite_values(X_train, num_features)

Column Quantity (In TON): Inf count = 0, NaN count = 166


In [49]:
num_features = [f for f in num_features if f not in ['year']]

In [52]:
num_features

['travel_distance',
 'Quantity (In TON)',
 'distance_per_ton',
 'route_frequency',
 'avg_route_price',
 'nifty_infra_price',
 'month_sin',
 'month_cos']

In [53]:
df[num_features]

,travel_distance,Quantity (In TON),distance_per_ton,route_frequency,avg_route_price,nifty_infra_price,month_sin,month_cos
0,1115,75.00,14.866667,1,4.575000e+05,6585.60,-5.000000e-01,0.866025
1,367,13.00,28.230769,3,4.996787e+04,6095.40,-8.660254e-01,0.500000
2,493,12.00,41.083333,1,4.712062e+04,6585.60,-5.000000e-01,0.866025
3,304,40.00,7.600000,4,2.268624e+05,6095.40,-8.660254e-01,0.500000
4,306,39.40,7.766497,4,2.217552e+05,6095.40,-8.660254e-01,0.500000
...,...,...,...,...,...,...,...,...
31077,625,35.79,17.462978,1,2.164178e+05,5738.70,1.224647e-16,-1.000000
31078,653,29.75,21.949580,1,1.729219e+05,5738.70,1.224647e-16,-1.000000
31079,399,42.00,9.500000,1,1.023758e+06,6115.35,-5.000000e-01,-0.866025
31080,399,30.00,13.300000,3,1.526042e+05,6115.35,-5.000000e-01,-0.866025


In [51]:
num_features.extend(['month_sin', 'month_cos'])  # Add cyclical features

In [21]:
df['period_dt'] = pd.to_datetime(df['period'], format='%B %Y')
df['year'] = df['period_dt'].dt.year
df['month'] = df['period_dt'].dt.month

In [22]:
df.head()

,start_pin,destination_pin,travel_distance,Quantity (In TON),period,nifty_infra_price,amount,distance_per_ton,route_frequency,avg_route_price,period_dt,year,month
0,30,2349,1115,75.0,November 2023,6585.6,457500.00,14.866667,1,457500.0000,2023-11-01,2023,11
1,42,233,367,13.0,October 2023,6095.4,81453.13,28.230769,3,49967.8700,2023-10-01,2023,10
2,42,257,493,12.0,November 2023,6585.6,47120.62,41.083333,1,47120.6200,2023-11-01,2023,11
3,42,325,304,40.0,October 2023,6095.4,212500.00,7.600000,4,226862.3775,2023-10-01,2023,10
4,43,218,306,39.4,October 2023,6095.4,233938.00,7.766497,4,221755.2500,2023-10-01,2023,10


In [23]:
num_features.extend(['year', 'month'])

In [55]:
num_features

['travel_distance',
 'Quantity (In TON)',
 'distance_per_ton',
 'route_frequency',
 'avg_route_price',
 'nifty_infra_price',
 'month_sin',
 'month_cos']

In [60]:
for feature in num_features:
    X_train[feature] = X_train[feature].replace([np.inf, -np.inf], np.nan)
    X_test[feature] = X_test[feature].replace([np.inf, -np.inf], np.nan)
    
    feature_mean = X_train[feature].mean()
    X_train[feature] = X_train[feature].fillna(feature_mean)
    X_test[feature] = X_test[feature].fillna(feature_mean)

In [61]:
scaler = StandardScaler()
X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_test[num_features] = scaler.transform(X_test[num_features])

In [62]:
check_infinite_values(X_train, num_features)

In [63]:
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))

In [64]:
num_start_pins = df['start_pin'].nunique()
num_dest_pins = df['destination_pin'].nunique()
start_embed_dim = min(8, int(np.power(num_start_pins, 0.25)))
dest_embed_dim = min(16, int(np.power(num_dest_pins, 0.25)))

In [65]:
start_pin_input = Input(shape=(1,), name='start_pin')
dest_pin_input = Input(shape=(1,), name='destination_pin')
numerical_input = Input(shape=(8,), name='numerical_data') 

In [66]:
start_embedding = Embedding(input_dim=num_start_pins + 1, output_dim=start_embed_dim)(start_pin_input)
dest_embedding = Embedding(input_dim=num_dest_pins + 1, output_dim=dest_embed_dim)(dest_pin_input)

In [67]:
start_flat = Flatten()(start_embedding)
dest_flat = Flatten()(dest_embedding)

In [68]:
x = Concatenate()([start_flat, dest_flat, numerical_input])

In [69]:
x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)

x = Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)

output = Dense(1)(x)

In [70]:
# Create and compile model
model = Model(inputs=[start_pin_input, dest_pin_input, numerical_input], outputs=output)
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

In [71]:
train_inputs = [
    X_train['start_pin'].values,
    X_train['destination_pin'].values,
    X_train[num_features].values
]

test_inputs = [
    X_test['start_pin'].values,
    X_test['destination_pin'].values,
    X_test[num_features].values
]

In [72]:
history = model.fit(
    train_inputs,
    y_train_scaled,
    validation_data=(test_inputs, y_test_scaled),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/100
778/778 [==============================] - 4s 3ms/step - loss: 2.7085 - val_loss: 1.5600
Epoch 2/100
778/778 [==============================] - 1s 2ms/step - loss: 1.8096 - val_loss: 1.4135
Epoch 3/100
778/778 [==============================] - 2s 2ms/step - loss: 1.4469 - val_loss: 1.4753
Epoch 4/100
778/778 [==============================] - 2s 2ms/step - loss: 1.2152 - val_loss: 0.8819
Epoch 5/100
778/778 [==============================] - 1s 2ms/step - loss: 1.1163 - val_loss: 0.6731
Epoch 6/100
778/778 [==============================] - 1s 2ms/step - loss: 1.0858 - val_loss: 0.6612
Epoch 7/100
778/778 [==============================] - 1s 2ms/step - loss: 1.0214 - val_loss: 1.7861
Epoch 8/100
778/778 [==============================] - 2s 2ms/step - loss: 0.9933 - val_loss: 0.6593
Epoch 9/100
778/778 [==============================] - 1s 2ms/step - loss: 0.9958 - val_loss: 8.9718
Epoch 10/100
778/778 [==============================] - 1s 2ms/step - loss: 0.9892 - val_lo

In [73]:
prediction_train = model.predict(train_inputs)

778/778 [==============================] - 1s 730us/step


In [74]:
predictions = model.predict(test_inputs)
predictions_original = y_scaler.inverse_transform(predictions)

195/195 [==============================] - 0s 1ms/step


In [75]:
predictions_original

array([[-1227218.9  ],
       [-2196611.5  ],
       [  245357.77 ],
       ...,
       [  199641.02 ],
       [  372787.06 ],
       [  -11032.676]], dtype=float32)

In [76]:
mse = mean_squared_error(y_test_scaled, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_scaled, predictions)
print("MSE:", mse)
print("RMSE:", rmse)
print("R^2:", r2)

MSE: 0.49994167630137354
RMSE: 0.7070655389010084
R^2: 0.5011394750163297


In [79]:
df.to_csv('data_n4.csv')